I. Create a directory in storage

In [ ]:
dbutils.fs.mkdirs("dbfs:/FileStore/agri_data_archi")

Out[44]: True

II. Read incoming file as dataframe

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, TimestampType


schema = StructType([
    StructField("GENOTYPES", StringType(), True),
    StructField("plant_height_30DAT_CM", DoubleType(), True),
    StructField("plant_height_60DAT_CM", DoubleType(), True),
    StructField("plant_height_90DAT_CM", DoubleType(), True),
    StructField("days_for_flower_initiation_DAYS", IntegerType(), True),
    StructField("no_primery_branches_Num", DoubleType(), True),
    StructField("fruit_length_cm", DoubleType(), True),
    StructField("fruit_circumference_mm", DoubleType(), True),
    StructField("avg_fruit_weight_gram", DoubleType(), True),
    StructField("no_fruits_plant", DoubleType(), True),
    StructField("fruit_TSS_Brix", DoubleType(), True),
    StructField("fruit_yield_plant_Kg", DoubleType(), True),
    StructField("yield_status", StringType(), True),
])



In [ ]:
yield_df = spark.read.option("mode","failFast").schema(schema).csv("dbfs:/FileStore/agridata/growth_and_growth_attribute.csv", header = True)

III. create and use database

In [ ]:
%sql
create database delta_growth_archi

In [ ]:
spark.sql("use delta_growth_archi")

Out[47]: DataFrame[]

Create tables for three tier architechture zones as bronze 1st, silver 2nd, and gold as 3rd. (specify location otherwise it will be stored in hive metastore)

In [ ]:

%sql
CREATE TABLE delta_growth_archi.plant_yield_data_bronze (
    GENOTYPES STRING,
    plant_height_30DAT_CM DOUBLE,
    plant_height_60DAT_CM DOUBLE,
    plant_height_90DAT_CM DOUBLE,
    days_for_flower_initiation_DAYS INT,
    no_primery_branches_Num DOUBLE,
    fruit_length_cm DOUBLE,
    fruit_circumference_mm DOUBLE,
    avg_fruit_weight_gram DOUBLE,
    no_fruits_plant DOUBLE,
    fruit_TSS_Brix DOUBLE,
    fruit_yield_plant_Kg DOUBLE,
    yield_status STRING,
    filename STRING,
    created_on TIMESTAMP
)
USING DELTA
LOCATION "dbfs:/FileStore/agri_data_archi/growth_bronze.delta"
PARTITIONED BY (yield_status)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [ ]:
%sql
CREATE TABLE delta_growth_archi.plant_yield_data_silver (
    GENOTYPES STRING,
    plant_height_30DAT_CM DOUBLE,
    plant_height_60DAT_CM DOUBLE,
    plant_height_90DAT_CM DOUBLE,
    fruit_length_cm DOUBLE,
    fruit_circumference_mm DOUBLE,
    avg_fruit_weight_gram DOUBLE,
    fruit_yield_plant_Kg DOUBLE,
    yield_status STRING,
    created_on TIMESTAMP,
    modified_on TIMESTAMP
)

USING DELTA
LOCATION "dbfs:/FileStore/agri_data_archi/growth_silver.delta"
PARTITIONED BY (yield_status)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [ ]:
%sql
CREATE TABLE delta_growth_archi.plant_yield_data_gold (
    GENOTYPES STRING,
    average_plant_hieght_CM DOUBLE,
    fruit_area_cm DOUBLE,
    fruit_yield_plant_Kg DOUBLE,
    yield_status STRING,
    last_updated TIMESTAMP
)
USING DELTA
LOCATION "dbfs:/FileStore/agri_data_archi/growth_gold.delta"
PARTITIONED BY (yield_status)
TBLPROPERTIES (delta.enableChangeDataFeed = true);

In [ ]:
display(spark.sql("show tables"))

database,tableName,isTemporary
delta_growth_archi,plant_yield_data,false
delta_growth_archi,plant_yield_data_bronze,false
delta_growth_archi,plant_yield_data_gold,false
delta_growth_archi,plant_yield_data_silver,false


Migration of data in sequence from bronze layer to gold layer (aggregation are based on business logic)

In [ ]:
%sql
-- copy command will load only new appended or added data instead of using load into or insert into since our bronze layer true source which will have all the info
COPY INTO delta_growth_archi.plant_yield_data_bronze
FROM (
  SELECT 
    cast(GENOTYPES as STRING),
    cast(plant_height_30DAT_CM as DOUBLE),
    cast(plant_height_60DAT_CM as DOUBLE),
    cast(plant_height_90DAT_CM as DOUBLE),
    cast(days_for_flower_initiation_DAYS as int),
    cast(no_primery_branches_Num as DOUBLE),
    cast(fruit_length_cm as DOUBLE),
    cast(fruit_circumference_mm as DOUBLE),
    cast(avg_fruit_weight_gram as DOUBLE),
    cast(no_fruits_plant as DOUBLE),
    cast(fruit_TSS_Brix as DOUBLE),
    cast(fruit_yield_plant_Kg as DOUBLE),
    cast(yield_status as STRING),
    input_file_name() as filename,
    current_timestamp() as created_on
  FROM 'dbfs:/FileStore/agridata'
)
FILEFORMAT = CSV
FORMAT_OPTIONS ('header' = 'true')
COPY_OPTIONS ('mergeSchema' = 'true');;



num_affected_rows,num_inserted_rows,num_skipped_corrupt_files
0,0,0


Track down for history

In [ ]:
%sql
-- in function we can see no affected rows beacuse of multiple copy command run (version 1)
-- second entry  with new_data (version 2)
describe history delta_growth_archi.plant_yield_data_bronze

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
2,2024-06-25T17:02:55.000+0000,4514929696160,faizanscf@gmail.com,COPY INTO,Map(),null,List(2011140080832843),0625-121832-cmnnvtxp,1,WriteSerializable,true,"Map(numFiles -> 4, numOutputRows -> 7, numOutputBytes -> 18703, numSkippedCorruptFiles -> 0)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-06-25T14:06:01.000+0000,4514929696160,faizanscf@gmail.com,COPY INTO,Map(),null,List(2011140080832843),0625-121832-cmnnvtxp,0,WriteSerializable,true,"Map(numFiles -> 0, numOutputRows -> 0, numOutputBytes -> 0, numSkippedCorruptFiles -> 0)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-06-25T14:01:59.000+0000,4514929696160,faizanscf@gmail.com,CREATE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [""yield_status""], properties -> {""delta.enableChangeDataFeed"":""true""})",null,List(2011140080832843),0625-121832-cmnnvtxp,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


In [ ]:

from pyspark.sql.functions import col
from delta.tables import DeltaTable

df1 = delta_table.history() \
    .select("operationMetrics.numOutputRows") \
    .where(col("operation") == "WRITE") \
    .orderBy(col("timestamp").desc()) \
    .take(2)

# Extract the number of output rows
list_no_rows = [int(i.numOutputRows) for i in df1]

# Compare the number of rows and print a message if necessary
if abs(list_no_rows[0] - list_no_rows[1]) >= 4:
    print("Please check your data")
else:
    pass

Instead of overwriting whole data use temp view and merge the selected data into silver table (this layer will as data warehouse for several use case for our purpose)

In [ ]:
%sql
-- version 1
CREATE OR REPLACE TEMPORARY VIEW updated_growth_data AS
SELECT * 
FROM 
  table_changes('delta_growth_archi.plant_yield_data_bronze', 1);



In [ ]:
%sql
-- version 2
CREATE OR REPLACE TEMPORARY VIEW updated_growth_data AS
SELECT * 
FROM 
  table_changes('delta_growth_archi.plant_yield_data_bronze', 2);


In [ ]:
%sql
SELECT * FROM delta_growth_archi.plant_yield_data_bronze VERSION AS OF 2;


GENOTYPES,plant_height_30DAT_CM,plant_height_60DAT_CM,plant_height_90DAT_CM,days_for_flower_initiation_DAYS,no_primery_branches_Num,fruit_length_cm,fruit_circumference_mm,avg_fruit_weight_gram,no_fruits_plant,fruit_TSS_Brix,fruit_yield_plant_Kg,yield_status,filename,created_on
G6,31.68,28.71,38.61,35,10.48,9.64,19.69,56.84,5.94,5.58,341.04,High,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000
G102,29.35,46.98,54.52,28,6.35,8.45,17.9,90.09,0.53,4.55,165.59,High,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000
G1,47.47,52.52,63.63,35,11.39,6.21,12.63,33.89,6.06,5.95,203.34,Medium,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000
G101,29.35,46.98,54.52,28,6.35,8.45,17.9,90.09,0.53,4.55,165.59,Medium,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000
G74,37.83,54.32,63.05,30,7.84,7.03,14.82,70.3,1.94,4.99,140.6,Low,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000
null,null,null,null,null,null,null,null,null,null,null,null,null,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000
null,null,null,null,null,null,null,null,null,null,null,null,null,dbfs:/FileStore/agridata/growth_and_growth_attribute-1.csv,2024-06-25T17:02:49.337+0000


In [ ]:
%sql
-- merging condintion for only new update and inserted data and remaining will be same 
MERGE INTO delta_growth_archi.plant_yield_data_silver PYDS
USING updated_growth_data UGDB
ON PYDS.GENOTYPES = UGDB.GENOTYPES
WHEN MATCHED THEN 
UPDATE 
SET 
    PYDS.plant_height_30DAT_CM = UGDB.plant_height_30DAT_CM, 
    PYDS.plant_height_60DAT_CM = UGDB.plant_height_60DAT_CM, 
    PYDS.plant_height_90DAT_CM = UGDB.plant_height_90DAT_CM, 
    PYDS.fruit_length_cm = UGDB.fruit_length_cm, 
    PYDS.fruit_circumference_mm = UGDB.fruit_circumference_mm, 
    PYDS.avg_fruit_weight_gram = UGDB.avg_fruit_weight_gram, 
    PYDS.fruit_yield_plant_Kg = UGDB.fruit_yield_plant_Kg, 
    PYDS.yield_status = UGDB.yield_status, 
    PYDS.created_on = current_timestamp()
WHEN NOT MATCHED THEN 
INSERT (
    GENOTYPES,
    plant_height_30DAT_CM,
    plant_height_60DAT_CM,
    plant_height_90DAT_CM,
    fruit_length_cm,
    fruit_circumference_mm,
    avg_fruit_weight_gram,
    fruit_yield_plant_Kg,
    yield_status,
    created_on,
    modified_on
) 
VALUES (
    GENOTYPES,
    plant_height_30DAT_CM,
    plant_height_60DAT_CM,
    plant_height_90DAT_CM,
    fruit_length_cm,
    fruit_circumference_mm,
    avg_fruit_weight_gram,
    fruit_yield_plant_Kg,
    yield_status,
    current_timestamp(), 
    current_timestamp()
);



num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
7,5,0,2


In [ ]:
%sql
select * from delta_growth_archi.plant_yield_data_silver

GENOTYPES,plant_height_30DAT_CM,plant_height_60DAT_CM,plant_height_90DAT_CM,fruit_length_cm,fruit_circumference_mm,avg_fruit_weight_gram,fruit_yield_plant_Kg,yield_status,created_on,modified_on


In [ ]:
%sql
describe history delta_growth_archi.plant_yield_data_silver

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
10,2024-06-25T17:25:50.000+0000,4514929696160,faizanscf@gmail.com,MERGE,"Map(predicate -> [""(GENOTYPES#30243 = GENOTYPES#30196)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2011140080832843),0625-121832-cmnnvtxp,9,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 13599, numTargetBytesRemoved -> 10590, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 5, executionTimeMs -> 8570, numTargetRowsInserted -> 2, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 2505, numTargetRowsUpdated -> 5, numOutputRows -> 7, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 4, numSourceRows -> 7, numTargetFilesRemoved -> 3, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 3716)",null,Databricks-Runtime/12.2.x-scala2.12
9,2024-06-25T17:25:14.000+0000,4514929696160,faizanscf@gmail.com,MERGE,"Map(predicate -> [""(GENOTYPES#28637 = GENOTYPES#28590)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2011140080832843),0625-121832-cmnnvtxp,8,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 13604, numTargetBytesRemoved -> 10590, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 5, executionTimeMs -> 10464, numTargetRowsInserted -> 2, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 3131, numTargetRowsUpdated -> 5, numOutputRows -> 7, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 4, numSourceRows -> 7, numTargetFilesRemoved -> 3, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 4871)",null,Databricks-Runtime/12.2.x-scala2.12
8,2024-06-25T17:24:33.000+0000,4514929696160,faizanscf@gmail.com,MERGE,"Map(predicate -> [""(GENOTYPES#27031 = GENOTYPES#26984)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2011140080832843),0625-121832-cmnnvtxp,7,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 13604, numTargetBytesRemoved -> 9939, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 5, executionTimeMs -> 11120, numTargetRowsInserted -> 2, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 3607, numTargetRowsUpdated -> 5, numOutputRows -> 7, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 4, numSourceRows -> 7, numTargetFilesRemoved -> 3, numTargetRowsNotMatchedBySourceDeleted -> 0, rewriteTimeMs -> 5323)",null,Databricks-Runtime/12.2.x-scala2.12
7,2024-06-25T17:24:06.000+0000,4514929696160,faizanscf@gmail.com,MERGE,"Map(predicate -> [""(GENOTYPES#25741 = GENOTYPES#25694)""], matchedPredicates -> [{""actionType"":""update""}], notMatchedPredicates -> [{""actionType"":""insert""}], notMatchedBySourcePredicates -> [])",null,List(2011140080832843),0625-121832-cmnnvtxp,6,WriteSerializable,false,"Map(numTargetRowsCopied -> 0, numTargetRowsDeleted -> 0, numTargetFilesAdded -> 4, numTargetBytesAdded -> 12736, numTargetBytesRemoved -> 0, numTargetDeletionVectorsAdded -> 0, numTargetRowsMatchedUpdated -> 0, executionTimeMs -> 5746, numTargetRowsInserted -> 7, numTargetRowsMatchedDeleted -> 0, scanTimeMs -> 1269, numTargetRowsUpdated -> 0, numOutputRows -> 7, numTargetDeletionVectorsRemoved -> 0, numTargetRowsNotMatchedBySourceUpdated -> 0, numTargetChangeFilesAdded -> 0, numSourceRows -

In [ ]:
%sql
INSERT OVERWRITE TABLE delta_growth_archi.plant_yield_data_gold
-- gold layer will be for dash boarding with high level of aggreagtion (serving layer for some use case)
SELECT 
    GENOTYPES, 
    AVG((plant_height_30DAT_CM + plant_height_60DAT_CM + plant_height_90DAT_CM) / 3) AS average_plant_height_CM, 
    SUM((fruit_circumference_mm / 10) * fruit_length_cm) AS fruit_area_cm, 
    SUM(fruit_yield_plant_Kg) AS total_fruit_yield_plant_Kg,
    yield_status, 
    current_timestamp() AS last_updated
FROM 
    delta_growth_archi.plant_yield_data_silver
GROUP BY 
    GENOTYPES, 
    yield_status;






num_affected_rows,num_inserted_rows
6,6


In [ ]:
%sql
describe history delta_growth_archi.plant_yield_data_gold

## lets work on one usecase where we update and add new data from backend with same name = 
####dbfs:/FileStore/agridata/growth_and_growth_attribute.csv
## analyse the version once again through migrating  data